In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint

In [ ]:
# make page
paper_size = '14x11 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
lines = []

node_sets = []
n_lines = 500
n_nodes_per_line = 40
y_start = 0
y_end = 14
x_start = 0
x_end = 10
node_x_centers = np.linspace(x_start, x_end, n_lines)
std_scale = 0.09
n_eval_points = 80

### initialize vals
node_ys = np.linspace(y_start, y_end, n_nodes_per_line)
centered_node_xs = np.zeros(node_ys.shape)
bez_eval_end_center = 1
bez_eval_end_noise = 0
bez_eval_end_limit = 1.
bez_eval_end_std_scale = 0.01

for i, node_x_center in enumerate(node_x_centers):
    new_x_noise = np.random.randn(n_nodes_per_line) * std_scale
    centered_node_xs = centered_node_xs + new_x_noise
    node_xs = node_x_center + centered_node_xs
    node_xs[:3] = node_x_center
    node_xs[-3:] = node_x_center
    nodes = np.asfortranarray([
        node_xs,
        node_ys,
        ])
    curve = bezier.Curve(nodes, degree=(nodes.shape[1]-1))
    eval_start = np.random.uniform(0, 0.03)
    eval_end = np.random.uniform(0.97, 1.)
    eval_points = np.linspace(eval_start, eval_end, n_eval_points)
    x, y = curve.evaluate_multi(eval_points)
    if i % 2:
        x = np.flipud(x)
        y = np.flipud(y)
    lines.append(np.stack([x, y]).T)
    node_sets.append(np.stack([node_xs, node_ys]).T)

In [ ]:
ls = [LineString(l) for l in lines]

mls = gp.make_like(gp.merge_LineStrings(ls), drawbox)

mask = drawbox

in_mask = mls.intersection(mask)

in_mask = sa.rotate(in_mask, -90)

split_point = 500

layer1 = in_mask[:split_point]
layer2 = in_mask[split_point:]

layers = []
layers.append(LineString(np.concatenate([np.array(l) for l in layer1])))
layers.append(LineString(np.concatenate([np.array(l) for l in layer2])))

# layers = [in_mask]

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesort')

sk.display(color_mode='none')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/246_500_lines.svg')

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=55
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
def oscillator(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v) + (a * v) + (b * u)
    dudt = np.cos(u) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

def oscillator2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v) + np.sin(u) + (a * v) + (b * u)
    dudt = np.cos(u) + np.cos(u) ** 2 + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v)  + np.cos(u * v) + (a * v) + (b * u)
    dudt = np.cos(u) + np.sin(u * v) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 500
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(.5, 4.5, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius
    y = np.sin(theta) * radius
    pts.append(Point(x, y))

In [ ]:
lfs = []


t_max = 3.7
t = np.linspace(0, t_max, 1801)
a = -0.4
b = 0.3
c = 0.75
d = -0.2



for pt in tqdm(pts):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lines)

sk.penWidth('0.3')
sk.vpype('linesimplify linesort ')

sk.display(color_mode='none')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/247_500_lines.svg')

# Try 2

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=55
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 500
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(.75, 3.45, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 3.3
    y = np.sin(theta) * radius + 0.5
    pts.append(Point(x, y))

In [ ]:
def ode2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(u * v + (a * v) + (b * u))
    dudt = np.cos(u) + np.sin(u * v) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
lfs = []


t_max = 2.7
t = np.linspace(0, t_max, 1801)
a = -0.2
b = -0.2
c = 0.04
d = -0.25



for pt in tqdm(pts):
    sol = odeint(ode2, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

In [ ]:
layers = []
layers.append(lines[:250])
layers.append(lines[250:])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/249_500_lines.svg')

# Try 3

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 3500
thetas = np.linspace(0, np.pi*14, n_lines)
radii = np.linspace(.5, 5.45, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 3.3
    y = np.sin(theta) * radius + 0.5
    pts.append(Point(x, y))

In [ ]:
def ode2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(u * v + (a * v) + (b * u))
    dudt = np.cos(u) + np.sin(u * v) + np.cos(c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
lfs = []


t_max = 2.7
t = np.linspace(0, t_max, 701)
a = -0.2
b = -0.25
c = 0.04
d = -0.25



for pt in tqdm(pts):
    sol = odeint(ode2, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

In [ ]:
lbs = lines.buffer(0.07, cap_style=2, join_style=2).boundary

In [ ]:
lbs = gp.merge_LineStrings([l for l in lbs if l.length > 0.9])

In [ ]:
n_layers = 1
layer_inds = np.split(np.arange(len(lbs)), n_layers)

In [ ]:
layers = []
for ind_set in layer_inds:
    layer = [lbs[i] for i in ind_set]
    layers.append(gp.merge_LineStrings(layer))
    

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/251_3500_lines.svg')